<a href="https://colab.research.google.com/github/yasirabd/solver-society-job-analysis/blob/main/2_3_4_Ekstrak_skill_(en_%26_id).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install spacy-langdetect -q
!pip install contextualSpellCheck -q
!python -m spacy download xx_ent_wiki_sm -q
!python -m spacy download en_core_web_lg -q
!python -m spacy download en_core_web_sm -q
!pip install spacy[lookups] -q
!pip install transformers -q

!pip install ipython-autotime

%load_ext autotime

✔ Download and installation successful
You can now load the model via spacy.load('xx_ent_wiki_sm')
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_lg')
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
time: 2.91 ms


> Warning: proses bisa berjam-jam

# Objectives
Ekstrak job position, hasil ekstraksi adalah:
1. **Pendidikan**: kualifikasi pendidikan
2. **Skill**: keahlian yang dibutuhkan

Data input yang dibutuhkan:
1. jobstreet_clean_tahap3.csv
2. Seluruh .py file pada folder hmmtagger

Data output yang dihasilkan:
1. jobstreet_clean_tahap4.csv

Note: proses membutuhkan waktu sangat lama

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re

time: 3.26 ms


In [3]:
jobstreet = pd.read_csv("/content/drive/My Drive/Data Loker/jobstreet_clean_tahap3.csv")
jobstreet.head()

,id_loker,pulau,provinsi,job_position,company_name,years_of_experience,years_of_experience_cat,job_description,average_processing_time,company_industry,company_size,work_environment_waktu_bekerja,work_environment_gaya_berpakaian,work_environment_gaya_berpakaian_contoh,work_environment_tunjangan,work_environment_bahasa,platform,pendidikan
0,1,jawa,dki jakarta,spv finance,PT Axindo Infotama,3,0 - 4 tahun,[SEP][SEP]Usia Max. 35 tahun[SEP]Pendidikan Mi...,6,Retail/Merchandise,51 - 200 pekerja,sabtu,formal,kemeja polos & dasi,"kesehatan, umum",indonesia,jobstreet,d3
1,2,jawa,dki jakarta,qualitative research executive,Perusahaan Dirahasiakan,3,0 - 4 tahun,RESPONSIBILITIES:[SEP]Brief Description of Dut...,tidak tersedia,tidak tersedia,tidak spesifik,tidak tersedia,tidak spesifik,tidak spesifik,tidak tersedia,tidak dicantumkan,jobstreet,s1
2,3,jawa,jawa tengah,field collection wonogiri,OTO Group,0,0 - 4 tahun,[SEP]Lingkup pekerjaan: [SEP]Menjaga account d...,14,Perbankan/Pelayanan Keuangan,lebih dari 5000 pekerja,senin - jumat,bisnis,kemeja,"kesehatan, bonus",indonesia,jobstreet,"d3, s1, sma"
3,4,jawa,dki jakarta,perawat di negara jerman,PT Jerman Manajemen,1,0 - 4 tahun,[SEP]Candidate must possess at least Diploma i...,11,Pendidikan,51 - 200 pekerja,sabtu - kamis,formal,kemeja polos & dasi,kesehatan,indonesia,jobstreet,d3
4,5,jawa,jawa tengah,field collection pati,OTO Group,0,0 - 4 tahun,[SEP]Lingkup pekerjaan: [SEP]Menjaga account d...,14,Perbankan/Pelayanan Keuangan,lebih dari 5000 pekerja,senin - jumat,bisnis,kemeja,"kesehatan, bonus",indonesia,jobstreet,"d3, s1, sma"


time: 255 ms


In [4]:
jobstreet.shape

(14036, 18)

time: 2.25 ms


# Ekstrak skill

In [5]:
job_description = jobstreet['job_description'].str.lower()

job_description.head()

0    [sep][sep]usia max. 35 tahun[sep]pendidikan mi...
1    responsibilities:[sep]brief description of dut...
2    [sep]lingkup pekerjaan: [sep]menjaga account d...
3    [sep]candidate must possess at least diploma i...
4    [sep]lingkup pekerjaan: [sep]menjaga account d...
Name: job_description, dtype: object

time: 38.9 ms


NLP tasks id/end corpus:
- detect language: id/en
- tokenization
- remove stopwords
- stemming/lemmatization
- part-of-speech tagging
- select only noun and adjective

In [6]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
nltk.download('stopwords')
from nltk.tokenize import word_tokenize
nltk.download('averaged_perceptron_tagger')

import spacy
from spacy_langdetect import LanguageDetector
from spacy.lang.en.stop_words import STOP_WORDS
from spacy.lang.id.stop_words import STOP_WORDS
# from spacy.lang.id import LOOKUP
import contextualSpellCheck

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
time: 2.6 s


In [7]:
lang_model = spacy.load('en_core_web_sm')
lang_model.add_pipe(LanguageDetector(), name="language_detector", last=True)

corpus_id = list()
corpus_en = list()
corpus_other = list()

for idx, desc in enumerate(job_description):
    # remove token [sep]
    d = ' '.join(desc.split('[sep]')).strip()

    # check language
    doc = lang_model(d)
    lang = doc._.language['language']

    if lang == 'id':
        corpus_id.append(desc)
    elif lang == 'en':
        corpus_en.append(desc)
    else:
        corpus_other.append(desc)

time: 12min 48s


In [8]:
print("banyak corpus indonesia:", len(corpus_id))
print("banyak corpus inggris:", len(corpus_en))
print("banyak corpus lain:", len(corpus_other))

banyak corpus indonesia: 7280
banyak corpus inggris: 6748
banyak corpus lain: 8
time: 1.8 ms


Lakukan langsung iterasi semua

In [9]:
# POS-tagging modules indonesia
from tagger import MainTagger
from tokenization import Tokenization

# initialize tagging model
lexicon_path = "/content/drive/My Drive/Data Loker/Lexicon.trn"
ngram_path = "/content/drive/My Drive/Data Loker/Ngram.trn"
tagger = MainTagger(lexicon_path, ngram_path, 0, 3, 3, 0, 0, False, 0.2, 0, 500.0, 1)
tokenize = Tokenization()

list_keywords = ['kemampuan', 'mampu', 'mengerti', 'memiliki', 'menguasai',
                 'mahir', 'pengalaman', 'disukai', 'menyukai', 'mempunyai',
                 'berkeahlian', 'requirement', 'qualification', 'required skill',
                 'skill', 'membuat', 'kompetensi', 'kompeten', 'operasional',
                 'punya', 'paham', 'bisa', 'diutamakan', 'menggunakan',
                 'bersedia', 'jago', 'pernah bekerja']

time: 308 ms


In [ ]:
lang_model = spacy.load('en_core_web_sm')
lang_model.add_pipe(LanguageDetector(), name="language_detector", last=True)

nlp_en = spacy.load('en_core_web_lg')
contextualSpellCheck.add_to_pipe(nlp_en)

id_counter, en_counter = 0, 0
result = list()

for idx,desc in enumerate(job_description):
    # remove token [sep]
    d = ' '.join(desc.split('[sep]')).strip()

    # check language
    doc = lang_model(d)
    lang = doc._.language['language']
    #######################################
    ## ENGLISH
    #######################################
    if lang == 'en':

        # spell check
        doc = nlp_en(d)
        result_spell_check = doc._.outcome_spellCheck
        
        doc = nlp_en(result_spell_check)
        # English pipeline
        skill_en = list()
        for word in doc:
            # jika punctuation
            if word.is_punct:
                continue
            
            # jika stop words
            lexeme = nlp_en.vocab[word.text]
            if lexeme.is_stop:
                continue

            # jika out-of-vocabulary
            if word.is_oov:
                continue
            
            # pilih noun
            if word.pos_ == 'NOUN':
                skill_en.append(word.lemma_)
        result.append(['en', skill_en])
        en_counter += 1

    #######################################
    ## INDONESIA
    #######################################
    elif lang == 'id':
        # split berdasarkan token [sep]
        split_desc = [data.strip() for data in desc.split('[sep]')]

        skill_id = list()
        for item in split_desc:
            # jika list keywords terdapat pada sentence
            if any(x in item for x in list_keywords):
                
                # remove punctuation
                item = re.sub(r'[^\w\s]', '', item)
                # remove extra space
                item = re.sub(' +', ' ', item) 

                # tagging kata
                output = tokenize.sentence_extraction(tokenize.cleaning(item))
                word_tagged = list()
                for o in output:
                    strtag = " ".join(tokenize.tokenisasi_kalimat(o)).strip()
                    word_tagged += [" ".join(tagger.taggingStr(strtag))]
                
                # collect skill dari tag 'NN' (Noun) dan 'JJ' (Adjective)
                word_selected = list()
                for word in word_tagged[0].split():
                    # cek tag
                    word, tag = word.split('/')[0], word.split('/')[1] 
                    if tag == "NN" or tag == "JJ":
                        word_selected.append(word)

                skill_id.append(word_selected)
        
        if len(skill_id) >= 1:
            # join list of skill
            s = list()
            for skill in skill_id:
                s += skill
            result.append(['id', s])
        else:
            result.append(['id', ['tidak dicantumkan']])
        id_counter += 1

    #######################################
    ## OTHER LANGUAGE
    #######################################
    else:
        result.append(['other', ['bahasa tidak termasuk']])
    # print(en_counter, id_counter)
    # if en_counter == 2 and id_counter == 2:
    #     break
    if idx != 0 and idx%500 == 0:
        print("selesai pada data ke-{}".format(idx) )